-/-/- - Author: Abdoulaye DIALLO  
-/-/- - École: ENSAI  
-/-/- - Cours: TAL

## Références:
### [Named Entity Recognition and Co-occurrence Graphs](https://www.milowski.com/journal/entry/2020-02-14T16:11:00-08:00/) Author Alexander milowski

### [Spacy](https://spacy.io/usage/linguistic-features)

In [5]:
import ssl
import urllib.request as web
import newspaper as np           # /!\ care with numpy alias
import feedparser as fp
import pandas as pd

import os
import json
import re
from time import perf_counter 
import time
import tqdm

In [4]:
! curl -O https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/lemonde.json 
! curl -O https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/osta/sources.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.3M  100 14.3M    0     0   926k      0  0:00:15  0:00:15 --:--:--  965k0:00:08  0:00:08  986k5  0:00:13  0:00:02 1033k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   916  100   916    0     0   2726      0 --:--:-- --:--:-- --:--:--  2726


In [2]:
os.getcwd()

'/Users/abdoulayediallo/Documents/Documents – MacPro/ENSAI/Cours/NLP/projets/web-scraping-lemonde'

In [5]:
ssl._create_default_https_context = ssl._create_unverified_context

# (b) recuperer le fichier RSS
url = "https://investir.lesechos.fr/RSS"
fn = "info-marches-investir-bourse-les-echos.xml"
data = fp.parse("/".join([url,fn]))

# (c) visualiser les elements du fichier RSS
print(data.feed.title)
print(" ")
print(data.feed.published)
print(" ")
# (d) iterer sur les entrees du flux RSS
for item in data.entries:
    print(item.title, item.published, item.link)
    break

Marchés, Indices, matières premières, analyses de séance, Taux
 
Wed, 10 Feb 2021 18:24:00 +0200
 
Paris Clôture : Les stocks de pétrole aux Etats-Unis sont venus chambouler la Bourse en toute fin de séance Wed, 10 Feb 2021T17:41:00 +0200 https://investir.lesechos.fr/marches/actualites/les-stocks-de-petrole-aux-etats-unis-sont-venus-chambouler-la-bourse-de-paris-en-toute-fin-de-seance-1948748.php?xtor=RSS-24


# ÉCOUTE RSS ET EXTRACTION

# TODO:
### 1.a Data load: lemonde
    - Lecture de la base de données existantes lemonde.json
    – regardez au passage la structure de ce fichier json et les informations qu’il contient

In [4]:
with open("data_in/lemonde.json", 'r') as l:
    lemonde = json.load(l)

print('Loaded', len(lemonde), "entries of lemonde")

Loaded 3041 entries of lemonde


### Structure du fichier

In [4]:
[(k, v) for k, v in enumerate(lemonde.items())][0] # Reveal on values of the data

(0,
 ('https://www.lemonde.fr/international/article/2020/12/06/l-ue-met-en-place-un-nouveau-regime-de-sanctions-transversales_6062390_3210.html',
  {'title': 'L’UE met en place un nouveau régime de sanctions transversales',
   'date': '2020-12-06T14:00:08',
   'author': ['Laurence Girard',
    'Taha Oudghiri',
    'Vice-Président De L Association Marocaine Des Économistes D Entreprises Ameen',
    'Stéphane Foucart',
    'Jean-Pierre Stroobants',
    'Bruxelles',
    'Bureau Européen'],
   'category': 'Europe',
   'content': 'L’opposant russe Alexeï Navalny, le 22 août 2019 à Moscou. Alexander Zemlianichenko / AP\n\nLe « nouveau régime de sanctions transversales » que les ministres des affaires étrangères de l’Union européenne devraient approuver lundi 7 décembre, à Bruxelles, va peut-être clore des années de discussion et ramener l’Union à son exact niveau d’ambition. Qui ne peut plus être vraiment celui du « pouvoir transformateur » dont rêvaient certains, mais celui d’un acteur de r

### 1.b Identifier les informations disponibles dans le flux RSS de l’exemple ci-dessus et les éléments correspondants dans la structure de données retournées par fp.parse

In [8]:
kys = []
df = {}
for keys, values in lemonde.items():
    for k, v in values.items():
        kys.append(k)
        df[k] = v
        
# Sanity check
len(kys) == len(lemonde.items())*len(set(kys))

True

In [9]:
# Les attributs disponible dans la base
set(kys)

{'author', 'category', 'content', 'date', 'image_file', 'image_link', 'title'}

### 1.c  Pour chacun des flux RSS listés dans le fichier sources.json :

    – Lire les données du flux RSS
    – Scanner les articles et repérer ceux qui ne sont pas deja présent dans la base de données

In [6]:
# load
with open("data_in/sources.json", 'r') as s:
    source = json.load(s)
print('Loaded', len(source), "entries of source")

Loaded 14 entries of source


#### scan

In [13]:
# new df
liens_manquants = []
for categories, links in source.items():
    parsedlinks = fp.parse(links)
    category = categories
        # update de la base existante
    for entry in parsedlinks.entries:
        link = entry.link
        if link not in lemonde.keys():
            liens_manquants.append(link)
            
# head des liens non existant dans la base de données

print("Il y a ",len(liens_manquants), "liens manquant.", "Sans doublons il y'a ", len(set(liens_manquants)), "\n")

liens_manquants = list(set(liens_manquants)) # On dédoublonne
print(liens_manquants[:5]) # Affichage des 5 premiers  items seulement
print(".."*10)

Il y a  273 liens manquant. Sans doublons il y'a  229 

['https://www.lemonde.fr/m-le-mag/article/2021/02/05/le-proces-des-colston-4-ravive-les-tourments-de-bristol-sur-son-passe-negrier_6068939_4500055.html', 'https://www.lemonde.fr/economie/article/2021/02/08/veolia-lance-les-hostilites-contre-suez-qui-se-rebiffe_6069168_3234.html', 'https://www.lemonde.fr/afrique/article/2021/02/10/negocier-avec-les-djihadistes-au-burkina-faso-une-option-de-moins-en-moins-taboue_6069462_3212.html', 'https://www.lemonde.fr/afrique/article/2021/02/10/ethiopie-80-du-tigre-est-inaccessible-pour-l-aide-humanitaire-selon-la-croix-rouge-locale_6069488_3212.html', 'https://www.lemonde.fr/emploi/article/2021/02/10/les-entreprises-confrontees-au-defi-de-la-remotivation-des-salaries_6069414_1698637.html']
....................


# TODO: 

## 2
    - Pour l’un des articles de votre flux RSS, regardez le code HTML de la page web. Pouvez-vous identifier les zones contenant le titre de l’article ? le texte de l’article ? En quoi le recours à la librairie newspaper3k est-il utile ? Quelles informations peut-on ainsi récupérer concernant un article ?TODO: 


**Réponse:**  
- Identification des zones contenant le titre de l’article
    - Le title est. dans `class="article__heading"`  
    - le text est dans `class="article__wrapper "`

- **Newspaper3k** permet de reconnaitre automatiquement les balise dans un documents html et de facilité le parsing des données et leur scraping.

- On peut récupere le caption des images dans `class="article__legend"`et le temps de lecture ncessaire dans `class="meta__reading-time meta__reading-time--header"`


## 3
Reprendre votre programme de scrapping et le compléter de maniere à mettre a jour votre base de données avec le texte des articles du flux RSS et le lien vers l’image illustrant l’article lorsqu’il y en a une.  
Vous pouvez inclure d’autres informations qui vous semblent utile pour un traitement ultérieur.

In [14]:
liens_existant = lemonde.keys() 
update_monde = lemonde.copy() # to not lose raw data

t1 = perf_counter()  
print("Length before updating data: ", len(update_monde))
# new df
for url in source.values():
    new_df = fp.parse(url)
    category = url[re.search(r'.fr/', url).start()+4:].split("/")[0] # retrieve new categories 
    
    # update de la base existante
    for item in new_df.entries:
        link = item.link
        time.sleep(3)
        if link not in liens_existant:
            article = np.Article(link)
            article.download()
            article.parse()
            update_monde[link] = {'title' : article.title,
                             'date' : article.publish_date.isoformat(), 
                             'author' : article.authors, 
                             'category' : category, 
                             'content' : article.text, 
                             'image_link' : article.top_image}
    t2 = perf_counter()

print("Length after updating data: ", len(update_monde))
print("Temps: ",round((t2-t1),1), "s")


Length before updating data:  3041
Length after updating data:  3271
Temps:  1037.3 s


In [32]:
with open("data_out/update_lemonde"+time.strftime("%Y%m%d-%H%M%S")+".json", 'w') as update_:
    json.dump(update_monde, update_)

# Wrap in function
Vous reformulerez le code de votre programme de scrapping de manière à en faire une fonction prenant en entrée la base de données dans son état actuel et la liste des flux à écouter et retournant la base de donéees mise à jour.    
   **En pratique**, comment utiliseriez-vous cette fonction (pensez à la sauvegarde des données, à la nécessité d’une mise à jour à intervalle régulier, etc.) ?

In [13]:
def jsonupdater(sourceslinks, df_to_update, sleeptime=0):
    """ 
    Input:
    ------
    sourceslinks: link to parse/scan for updating df_to_update: dict variables where keys are links (in our case)
                  change it to list using *sourceslinks if you've a list of link.
    df_to_update: json file containing to update
    sleeptime: (optional) time to wait for next data retrieval from website.
               lemonde.fr/robots.txt) ne précise pas de sleeptime, il faut tester plusieurs sleeptime si vous avez 403.
    
    Output:
    -------
    df_to_update: json file updated
    failledlink: failled link to download
    """

    print("Length before updating data: ", len(df_to_update))
    # time.sleep(sleeptime) # to be edited at your convenience, deaful est à 0, some bots may denied your multiplied request.

    failledlink = []
    for url in tqdm.tqdm(sourceslinks.values()):
        new_df = fp.parse(url)
        category = url[re.search(r'.fr/', url).start()+4:].split("/")[0] # retrieve new categories 

        # update de la base existante
        for item in new_df.entries:
            link = item.link
            time.sleep(sleeptime)
            liens_existant = df_to_update.keys() # Lien issue et existant dans notre base disponible, lemonde.
            if link not in liens_existant:
                article = np.Article(link)
                article.download()
                if article.download_state != 2: # controle des liens qui échouent
                    print("URL non téléchargé: ", link)
                    failledlink.append(url)
                    continue
                article.parse()
                df_to_update[link] = {'title' : article.title,
                                 'date' : article.publish_date.isoformat(), 
                                 'author' : article.authors, 
                                 'category' : category, 
                                 'content' : article.text, 
                                 'image_link' : article.top_image}
    
    # saving updated dataset name include datetime
    with open("data_out/update_lemonde"+time.strftime("%Y%m%d-%H%M%S")+".json", 'w') as jsonfile:
        json.dump(df_to_update, jsonfile)
    
    print("Length after updating data: ", len(df_to_update))    
    
    return df_to_update, failledlink

In [14]:
update_monde, failled_links = jsonupdater(source, lemonde, sleeptime=1)

  0%|          | 0/14 [00:00<?, ?it/s]

Length before updating data:  3041


100%|██████████| 14/14 [06:57<00:00, 29.80s/it]

Length after updating data:  3277


# Extraction d’information

In [15]:
# temp reload in case kernel failled
with open("data_out/update_lemonde20210210-215757.json", 'r') as data:
    update_monde = json.load(data)  
print('Loaded updated data', len(update_monde), "entries of source")

Loaded updated data 3271 entries of source


In [3]:
import spacy

# load pipeline and related models
nlp = spacy.load("fr_core_news_md")

### Sur la base lemonde

In [19]:
import random
%time
random.seed(142)
randomlink = random.choice(list(update_monde.keys()))
entities = nlp(update_monde[randomlink].get("content")).ents
[(entity.text, entity.start_char, entity.end_char, entity.label_)for entity in entities if(entity.label_ in ["LOC","PER","ORG"])]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


[('Laetitia Avia', 0, 13, 'PER'),
 ('Paris', 43, 48, 'LOC'),
 ('AFP', 90, 93, 'ORG'),
 ('La République en marche', 125, 148, 'ORG'),
 ('LRM', 150, 153, 'ORG'),
 ('Laetitia Avia', 156, 169, 'PER'),
 ('Paris', 199, 204, 'LOC'),
 ('Agence France', 341, 354, 'ORG'),
 ('Presse', 355, 361, 'ORG'),
 ('AFP', 363, 366, 'ORG'),
 ('Paris', 431, 436, 'LOC'),
 ('AFP', 452, 455, 'ORG'),
 ('BRDP', 634, 638, 'ORG'),
 ('Mediapart', 818, 827, 'PER'),
 ('Paris', 850, 855, 'LOC'),
 ('PD', 1227, 1229, 'ORG'),
 ('Mediapart', 1293, 1302, 'PER'),
 ('Joint', 1466, 1471, 'PER'),
 ('AFP', 1478, 1481, 'ORG'),
 ('Me Basile Ader', 1507, 1521, 'PER'),
 ('Mme Avia', 1876, 1884, 'PER'),
 ('Assemblée', 2056, 2065, 'ORG'),
 ('LRM', 2085, 2088, 'ORG'),
 ('Palais', 2153, 2159, 'LOC'),
 ('Bourbon', 2160, 2167, 'LOC'),
 ('LRM', 2293, 2296, 'ORG'),
 ('Laetitia Avia', 2298, 2311, 'PER'),
 ('Conseil constitutionnel', 2438, 2461, 'ORG'),
 ('Avia', 2545, 2549, 'ORG'),
 ('Conseil', 2601, 2608, 'ORG'),
 ('Le Monde', 2626, 2634, 'O

In [84]:
randomlink = random.choice(list(update_monde.keys()))
test = update_monde[randomlink].get("content")
test

for doc in nlp.pipe(test, disable=["tagger", "parser"]):
    # Do something with the doc here
    print([(ent.text, ent.label_) for ent in doc.ents])
    break

[]


In [47]:
from spacy import displacy
op = {"ents": ["PER","LOC","ORG"]}
displacy.render(entities, style='ent', jupyter=True, options=op)

# TODO
### **6.**  
En vous référant à la documentation de spaCy, quelle(s) technique(s) de détection des entités nommées spaCy utilise-t-il ?  
À quoi correspondent les étiquettes IOB utilisées ?

**Spacy technique**  
Spacy utilise ces pipelines (XX_core_news_YY) ou `XX` et `YY`représentent respectivement le langage et la version(large/medium/). Il comprend des outils et Composants: tok2vec, tagger, parser, senter, ner(`éntitité nommées`), attribute_ruler, lemmatizer.  

**IOB**:  
-**I** - le mot est à l'intérieur d'une entité.
-**O** - le mot est en dehors d'une entité.
-**B** - mot est le début d'une entité.

techniquement on `token.ent_iob` qui indique si une entité l'intérieur, à l'extérieur ou au début d'une expression. Si aucun type d'entité n'est défini sur une entité, elle renverra une chaîne vide.

### **7**
Écrire un programme qui traite l’ensemble des textes que vous aurez chargé depuis le fichier lemonde.json et stock pour chaque article les entités trouvées et les informations afférentes. On cherchera à optimiser le temps de traitement en utilisant la commande **pipe** et en n’activant dans la chaîne de traitement spaCy uniquement les traitemens nécessaires pour la tâche à effectuer.

In [32]:
entites_corpus = {}
keys = list(update_monde.keys())
for element in keys:
    doc = nlp(update_monde[element]["content"])

    entites_doc = {}
    for entity in doc.ents:
        print(entity)

<class 'spacy.tokens.doc.Doc'>


In [69]:
import tqdm

In [92]:
assert len(update_monde) != 0
infos_articles = {}
for url in tqdm.tqdm([lien for lien in update_monde.keys()]):
    docs = nlp(update_monde[url]["content"], disable=["tagger", "parser"])
    info_article = {}
    for doc in docs.ents: 
        # temp=[doc for doc in docs.ents if doc.label_ in ["PER","LOC","ORG"]] # filtre sur les 3 seul labels précisé sur l'exercice
        if doc.label_ in ["LOC","ORG","PER"] : 
            info_article = {"mot" : doc.text, "entite" : doc.label_,
                           "debut" : doc.start_char, "fin" : doc.end_char
                          }
    if len(info_article) != 0:
        infos_articles[url] = info_article
    else:
        print("Aucune entité (LOC, ORG, PER) trouvé dans l'url :: {} ".format(url), end='\r')

# saving local
with open("data_out/infos_articles"+time.strftime("%Y%m%d-%H%M%S")+".json", 'w') as jsonfile:
    json.dump(infos_articles, jsonfile)

  9%|▊         | 279/3271 [00:07<01:19, 37.52it/s]

 39%|███▉      | 1270/3271 [00:36<00:52, 38.42it/s]

 40%|████      | 1324/3271 [00:37<00:45, 43.13it/s]

 44%|████▍     | 1445/3271 [00:40<00:48, 37.96it/s]

 77%|███████▋  | 2509/3271 [01:11<00:22, 34.23it/s]

100%|██████████| 3271/3271 [01:32<00:00, 35.46it/s]


## Analyse des entités nomées
On cherche maintenant à analyser les entités nommées d ́etect ́ees à l'étape précédente, notamment leur relations.
# TODO :
**7.** Pour chaque type d’entité, déterminer les 20 plus fréquentes dans la collection de documents. Commentez le résultat obtenu. On mémorisera ces entités dans trois listes distinctes (une par type d’entité).

In [17]:
# temp reload in case kernel failled or next day work
with open("data_out/infos_articles20210215-001429.json", 'r') as data:
    articles = json.load(data)

In [46]:
# Let's use the same data set we've saved as article to count occurences

# Create an empty dict
dico_occurence = {"PER":{},"LOC":{},"ORG":{}}
# count occurence while they're added in dict
for label in articles.values():
    #print(label.get("mot"))
    if label.get("mot") not in dico_occurence[label.get("entite")]:
        dico_occurence[label.get("entite")][label.get("mot")]=1
    else:
        dico_occurence[label.get("entite")][label.get("mot")]+=1
        
        
# sort occurence in nested dict        
order_occurences = {key : dict(sorted(val.items(), key = lambda ele: ele[1], reverse=True)) for key, val in dico_occurence.items()} 

# get 20 first occurences
PER, LOC, ORG = list(order_occurences.get('PER'))[:20], list(order_occurences.get('LOC'))[:20], list(order_occurences.get('ORG'))[:20]

In [47]:
print("Par ordre de fréquence: ","\n","\n", PER, "\n","\n", LOC, "\n","\n", ORG)

Par ordre de fréquence:  
 
 ['Donald Trump', 'Joe Biden', 'Emmanuel Macron', 'Jean Castex', 'Marine Jeannin(Accra', 'Jean-Michel Blanquer', 'Olivier Véran', 'Gérald Darmanin', 'Nicolas Sarkozy', 'Tesla', 'Bruno Le Maire', 'Trump', 'M. Trump', 'Moretti', 'Boris Johnson', 'BioNtech', 'Mike Pence', 'Olivier Duhamel', 'M. Biden', 'Jan Egeland'] 
 
 ['Etat', 'France', 'la France', 'Etats-Unis', 'Paris', 'Etats', 'Europe', 'Français', 'Uni', 'Chine', 'Allemagne', 'Pékin', 'Capitole', 'Américains', 'la Chine', 'Russie', 'Algérie', 'Iran', 'Afrique', 'Berlin'] 
 
 ['AFP', 'Le Monde', 'Reuters', 'AP', 'LRM', 'UE', 'Etat', 'Monde', 'Sénat', 'Bourse', 'Union européenne', 'Parlement', 'Commission', 'Insee', 'Union', 'Res Publica', 'New York Times', 'Assemblée nationale', 'Commission européenne', 'Total']


In [48]:
order_occurences

{'PER': {'Donald Trump': 24,
  'Joe Biden': 13,
  'Emmanuel Macron': 13,
  'Jean Castex': 6,
  'Marine Jeannin(Accra': 5,
  'Jean-Michel Blanquer': 5,
  'Olivier Véran': 5,
  'Gérald Darmanin': 4,
  'Nicolas Sarkozy': 4,
  'Tesla': 4,
  'Bruno Le Maire': 4,
  'Trump': 4,
  'M. Trump': 3,
  'Moretti': 3,
  'Boris Johnson': 3,
  'BioNtech': 3,
  'Mike Pence': 3,
  'Olivier Duhamel': 3,
  'M. Biden': 3,
  'Jan Egeland': 2,
  'Xi Jinping': 2,
  'Robert': 2,
  'Thierry Herzog': 2,
  'Macron': 2,
  'Freud': 2,
  'Vladimir Poutine': 2,
  'Marie': 2,
  'Mme de Montchalin': 2,
  'Inès Derbak': 2,
  'Laurence Caramel': 2,
  'Amoako Boafo': 2,
  'Benoît Payan': 2,
  'Matt Hancock': 2,
  'Jésus': 2,
  'Michèle Rubirola': 2,
  'Tweet': 2,
  'Marine Le Pen': 2,
  'Huawei': 2,
  'Eliott Brachet(Khartoum': 2,
  'M. Véran': 2,
  'M. Chenet': 2,
  'Blanquer': 2,
  'Laetitia Avia': 2,
  'Mme Borne': 2,
  'Odile Jacob': 2,
  'Eric Zemmour': 2,
  'Elena Pougin': 2,
  'François Léotard': 2,
  'Patrick Isoir

# TODO:
**8.** Érire une fonction qui prend en entrée une paire d’entités et les types correspondant et retourne le nombre de co-occurrences de ces deux entités au sein d’un même document dans la collection.  


In [60]:
import itertools

def coOccurence(ent1, ent2):
    """
    Input: Two list of occurences (20 first occurences)
    Output: co_occurence matrix in pandas pivot table format
    
    """
    nestedlist = [i for i in [ent1,ent2]]
    combi = [list(itertools.combinations(i,2)) for i in nestedlist] # touttes les combinaisons possible
    tmp = list(itertools.chain.from_iterable((i, i[::-1]) for c_ in combi for i in c_)) # flatten list et re-ordonnée de maniere inversé
    df = pd.DataFrame(tmp) # init df avec tmp
    coOcc = pd.pivot_table(df, index=0, columns=1, aggfunc='size', fill_value=0)
    return tmp, coOcc

# See https://stackoverflow.com/questions/42814452/co-occurrence-matrix-from-nested-list-of-words/42814963#42814963

pd.set_option('display.max_columns', 40) # Rappel 20 occ plus fredquents gardés
pd.set_option('display.max_rows', 40)
pd.set_option('display.expand_frame_repr', True)

# co_occurence
_, coOcc = coOccurence(LOC, ORG)
coOcc

1,AFP,AP,Afrique,Algérie,Allemagne,Américains,Assemblée nationale,Berlin,Bourse,Capitole,Chine,Commission,Commission européenne,Etat,Etats,Etats-Unis,Europe,France,Français,Insee,Iran,LRM,Le Monde,Monde,New York Times,Paris,Parlement,Pékin,Res Publica,Reuters,Russie,Sénat,Total,UE,Uni,Union,Union européenne,la Chine,la France
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AFP,0,1,0,0,0,0,1,0,1,0,0,1,1,1,0,0,0,0,0,1,0,1,1,1,1,0,1,0,1,1,0,1,1,1,0,1,1,0,0
AP,1,0,0,0,0,0,1,0,1,0,0,1,1,1,0,0,0,0,0,1,0,1,1,1,1,0,1,0,1,1,0,1,1,1,0,1,1,0,0
Afrique,0,0,0,1,1,1,0,1,0,1,1,0,0,1,1,1,1,1,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,1
Algérie,0,0,1,0,1,1,0,1,0,1,1,0,0,1,1,1,1,1,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,1
Allemagne,0,0,1,1,0,1,0,1,0,1,1,0,0,1,1,1,1,1,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,1
Américains,0,0,1,1,1,0,0,1,0,1,1,0,0,1,1,1,1,1,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,1
Assemblée nationale,1,1,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,1,0,1,1,1,1,0,1,0,1,1,0,1,1,1,0,1,1,0,0
Berlin,0,0,1,1,1,1,0,0,0,1,1,0,0,1,1,1,1,1,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,1
Bourse,1,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,1,0,1,1,1,1,0,1,0,1,1,0,1,1,1,0,1,1,0,0


#### Sur les 3 istes précédentes

**9.** En partant des listes de la question 8, on utilisera la fonction de la question précédente pour chercher les paires d’entités de type différents qui apparaissent le plus souvent ensemble. Commentez les résultats retournés.

In [63]:
from collections import Counter 

def groupCoocc(dict_):
    """
    Input: dictionnary of occurences with type of occurence
    
    Output: counted ordered dict occurences
    """
    count_diff_occ = {}

    for k, v in dict_.items():

        # dico occurence pour chaque key
        result = dict(Counter(v.keys()))
        for key in result :
            if not key in count_diff_occ :
                count_diff_occ[key] = result[key]
            else :
                count_diff_occ[key] = count_diff_occ[key] + result[key]
    # sorting
    sort_counted_dicoocc = sorted(list(count_diff_occ.items()), key = lambda x:x[1], reverse = True)#[:20] # take 20 first items
                
    return sort_counted_dicoocc

In [64]:
groupCoocc(order_occurences)

[('Elysée', 3),
 ('Macron', 2),
 ('Addo', 2),
 ('REP', 2),
 ('BioNTech', 2),
 ('Bernard', 2),
 ('Etat', 2),
 ('Europe', 2),
 ('Lire', 2),
 ('Bourse', 2),
 ('Xerfi', 2),
 ('Donald Trump', 1),
 ('Joe Biden', 1),
 ('Emmanuel Macron', 1),
 ('Jean Castex', 1),
 ('Marine Jeannin(Accra', 1),
 ('Jean-Michel Blanquer', 1),
 ('Olivier Véran', 1),
 ('Gérald Darmanin', 1),
 ('Nicolas Sarkozy', 1),
 ('Tesla', 1),
 ('Bruno Le Maire', 1),
 ('Trump', 1),
 ('M. Trump', 1),
 ('Moretti', 1),
 ('Boris Johnson', 1),
 ('BioNtech', 1),
 ('Mike Pence', 1),
 ('Olivier Duhamel', 1),
 ('M. Biden', 1),
 ('Jan Egeland', 1),
 ('Xi Jinping', 1),
 ('Robert', 1),
 ('Thierry Herzog', 1),
 ('Freud', 1),
 ('Vladimir Poutine', 1),
 ('Marie', 1),
 ('Mme de Montchalin', 1),
 ('Inès Derbak', 1),
 ('Laurence Caramel', 1),
 ('Amoako Boafo', 1),
 ('Benoît Payan', 1),
 ('Matt Hancock', 1),
 ('Jésus', 1),
 ('Michèle Rubirola', 1),
 ('Tweet', 1),
 ('Marine Le Pen', 1),
 ('Huawei', 1),
 ('Eliott Brachet(Khartoum', 1),
 ('M. Véran',

In [67]:
# Cherchons les paires d’entités de type différents qui apparaissent le plus souvent ensemble


def base_occ(dico_occurence, order_occurences):
    """
    Input: 
        dico_occurence: dictionary with all occurence and label 
        order_occurences: dico_occurence ordered desc
    """
    DicoOcc = {}       
    tmp = list(order_occurences.keys()) # ['PER', 'LOC', 'ORG']
    for k1 in order_occurences.keys():
        # print( key) # PER
        for k2 in tmp:
            for firstk in order_occurences[k1]: 
                for secondk in order_occurences[k2]:
                    DicoOcc = {**groupCoocc(order_occurences), **coOccurence(k1, k2)}
    return groupCoocc

In [68]:
base_occ(dico_occurence, order_occurences)

TypeError: unhashable type: 'list'

In [ ]:
# Ici élément est le sous dico
For élément in dico:
   # Ici Key est sous_key
   Fo key in élément :
        # Si le mot est nouveau
        If not Key in dicoOcc :
            DicoOcc[Key] = 1
         # Si le mot déjà dans occ
         Else :
            DicoOcc [Key] += 1

# À faire :
**10.** Parmi les relations correspondant aux paires d’entités dont la fr ́equence de co-occurrence est élevée (cf . question 9), on cherche à en trouver quelques unes qui peuvent Être caractérisée par un verbe. Pour cela, écrivez une fonction qui prend en entrée deux entités et leurs types respectif et qui analyse l’ensemble des documents où ces entités apparaissent dans la même phrase de manière à établir la liste des verbes qui apparaissent dans ces phrase.  

**11.** En utilisant la fonction de la question précédente, d ́eterminez quelques couples d’entités et un verbe associé qui vous paraissent faire sens.  